In [1]:
import os
import torch
import random
import datetime
import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, get_svd_encoder
from src.embeddings import RecsysEmbedding

experiment_name = 'AGGSlatewiseGRU'
device = 'cuda:0'
seed = 123
pkl_path = '../pkl/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
torch.__version__

'1.12.1'

# Модель

In [3]:
import torch.nn.functional as F
torch.autograd.set_detect_anomaly(True)

class SessionwiseGRU(torch.nn.Module):
    def __init__(self, embedding, output_dim=1, dropout = 0.1):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.rnn_layer = torch.nn.GRU(
            input_size = embedding.embedding_dim, 
            hidden_size = embedding.embedding_dim, 
            batch_first = True,
            dropout=dropout
        )
        self.out_layer = torch.nn.Linear(embedding.embedding_dim, output_dim)


    def forward(self, batch):
        shp = batch['slates_item_indexes'].shape
        slate_size = shp[-1]
        # item_embs: batch, session, slate, embedding
        # user_embs: batch, session, embedding
        item_embs, user_embs = self.embedding(batch)
        
        session_length = item_embs.shape[-3]
        slate_size = item_embs.shape[-2]
        hidden = user_embs[None, ..., 0, :].contiguous()
        preds = []

        for rec in range(session_length):
            rnn_out, hidden = self.rnn_layer(
                item_embs[..., rec, :, :],
                hidden,
            )
            preds.append(rnn_out[..., None, :, :])
            hinned = rnn_out.mean(dim=1)

        preds = torch.cat(preds, axis=1)

        return self.out_layer(preds).reshape(shp)

In [4]:
a = torch.arange(30).reshape(2,3,5)
a.shape

torch.Size([2, 3, 5])

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [5]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = SessionwiseGRU(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
    output_dim=1
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device=device, lr=1e-3, num_epochs=5000, dummy=True,
    silent=True,
)


biulding affinity matrix...


3it [00:00, 2983.85it/s]
/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.4000000059604645, 'roc-auc': 0.3333333134651184, 'accuracy': 0.25}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.5 | f1: 0.5 | auc: 0.3333333134651184 | treshold: 0.51
Test: accuracy: 0.5 | f1: 0.5 | auc: 0.3333333134651184 | 
Val update: epoch: 2 |accuracy: 0.25 | f1: 0.4000000059604645 | auc: 0.6666666269302368 | treshold: 0.51
Test: accuracy: 0.25 | f1: 0.4000000059604645 | auc: 0.6666666269302368 | 
Val update: epoch: 3 |accuracy: 0.5 | f1: 0.5 | auc: 1.0 | treshold: 0.52
Test: accuracy: 0.5 | f1: 0.5 | auc: 1.0 | 
Val update: epoch: 4 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.53
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


(SessionwiseGRU(
   (embedding): RecsysEmbedding(
     (item_embeddings): Embedding(5, 32)
   )
   (rnn_layer): GRU(32, 32, batch_first=True, dropout=0.1)
   (out_layer): Linear(in_features=32, out_features=1, bias=True)
 ),
 {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0})

# ContentWise

In [6]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=150)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

20216 data points among 108 batches


In [7]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    
    model = SessionwiseGRU(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings
        ),
        output_dim=1
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)


Evaluating AGGSlatewiseGRU with svd embeddings


/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.16523703932762146, 'roc-auc': 0.4882444739341736, 'accuracy': 0.29794561862945557}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.09650824218988419 | f1: 0.176028311252594 | auc: 0.5079079866409302 | treshold: 0.01
Test: accuracy: 0.09963187575340271 | f1: 0.1812095046043396 | auc: 0.5190438032150269 | 
Val update: epoch: 1 |accuracy: 0.09650824218988419 | f1: 0.176028311252594 | auc: 0.533918023109436 | treshold: 0.01
Test: accuracy: 0.09963187575340271 | f1: 0.1812095046043396 | auc: 0.5390003323554993 | 
Val update: epoch: 2 |accuracy: 0.09650824218988419 | f1: 0.176028311252594 | auc: 0.5763778686523438 | treshold: 0.01
Test: accuracy: 0.09963187575340271 | f1: 0.1812095046043396 | auc: 0.5727401971817017 | 
Val update: epoch: 3 |accuracy: 0.854554295539856 | f1: 0.2241906374692917 | auc: 0.6165072321891785 | treshold: 0.08
Test: accuracy: 0.852140486240387 | f1: 0.220151886343956 | auc: 0.6077097654342651 | 
Val update: epoch: 4 |accuracy: 0.853158175945282 | f1: 0.22602634131908417 | auc: 0.6270208358764648 | treshold: 0.12
Test: accuracy: 0.8507599830627441 | f1: 0.2229092

/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.19434545934200287, 'roc-auc': 0.5647405385971069, 'accuracy': 0.48987650871276855}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.09650824218988419 | f1: 0.176028311252594 | auc: 0.6249620914459229 | treshold: 0.05
Test: accuracy: 0.09963187575340271 | f1: 0.1812095046043396 | auc: 0.6253974437713623 | 
Val update: epoch: 1 |accuracy: 0.8763630390167236 | f1: 0.21115799248218536 | auc: 0.6574681401252747 | treshold: 0.11
Test: accuracy: 0.8730702996253967 | f1: 0.2065509855747223 | auc: 0.6533944606781006 | 
Val update: epoch: 2 |accuracy: 0.7309319972991943 | f1: 0.26128706336021423 | auc: 0.6858200430870056 | treshold: 0.13
Test: accuracy: 0.7237560749053955 | f1: 0.26062771677970886 | auc: 0.6773802042007446 | 
Val update: epoch: 3 |accuracy: 0.7396907806396484 | f1: 0.2758084833621979 | auc: 0.7020728588104248 | treshold: 0.15000000000000002
Test: accuracy: 0.7304357886314392 | f1: 0.2726690173149109 | auc: 0.6918322443962097 | 
Val update: epoch: 4 |accuracy: 0.8419157862663269 | f1: 0.27391156554222107 | auc: 0.7113100290298462 | treshold: 0.14
Test: accuracy: 0.83860290050

In [8]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# RL4RS

In [9]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

45942 data points among 106 batches


In [10]:
for embeddings in ['explicit', 'neural', 'svd']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = SessionwiseGRU(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings,
            embedding_dim=40
        ),
        output_dim=1
    ).to(device)


    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)


Evaluating AGGSlatewiseGRU with explicit embeddings


/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.546219527721405, 'roc-auc': 0.41794049739837646, 'accuracy': 0.457526296377182}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.7678856253623962 | f1: 0.8209414482116699 | auc: 0.8182042241096497 | treshold: 0.43
Test: accuracy: 0.7606819272041321 | f1: 0.816969633102417 | auc: 0.8090521097183228 | 
Val update: epoch: 1 |accuracy: 0.7720940113067627 | f1: 0.844692051410675 | auc: 0.8363478779792786 | treshold: 0.39
Test: accuracy: 0.7699189782142639 | f1: 0.8438602685928345 | auc: 0.8244601488113403 | 
Val update: epoch: 2 |accuracy: 0.7909108400344849 | f1: 0.8495134711265564 | auc: 0.8488796949386597 | treshold: 0.35000000000000003
Test: accuracy: 0.7850078344345093 | f1: 0.8464660048484802 | auc: 0.8364692330360413 | 
Val update: epoch: 3 |accuracy: 0.7691916823387146 | f1: 0.8441577553749084 | auc: 0.8511664867401123 | treshold: 0.38
Test: accuracy: 0.7678152322769165 | f1: 0.8440828919410706 | auc: 0.837266206741333 | 
Val update: epoch: 4 |accuracy: 0.7817927002906799 | f1: 0.8487003445625305 | auc: 0.851799488067627 | treshold: 0.38
Test: accuracy: 0.7802442312240601 | f

...


Evaluating AGGSlatewiseGRU with svd embeddings
Test before learning: {'f1': 0.09881478548049927, 'roc-auc': 0.478157103061676, 'accuracy': 0.37855157256126404}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.6442219614982605 | f1: 0.7836191654205322 | auc: 0.6331443786621094 | treshold: 0.01
Test: accuracy: 0.6497400403022766 | f1: 0.7876877784729004 | auc: 0.6238398551940918 | 
Val update: epoch: 1 |accuracy: 0.6442219614982605 | f1: 0.7836191654205322 | auc: 0.6977781057357788 | treshold: 0.17
Test: accuracy: 0.6497400403022766 | f1: 0.7876877784729004 | auc: 0.6887626647949219 | 
Val update: epoch: 2 |accuracy: 0.6444395780563354 | f1: 0.7836529016494751 | auc: 0.7693146467208862 | treshold: 0.37
Test: accuracy: 0.6497884392738342 | f1: 0.7876299619674683 | auc: 0.7599225044250488 | 
Val update: epoch: 3 |accuracy: 0.735185980796814 | f1: 0.8147784471511841 | auc: 0.7960211038589478 | treshold: 0.39
Test: accuracy: 0.7297062277793884 | f1: 0.8120080828666687 | auc: 0.7845283150672913 | 
Val update: epoch: 4 |accuracy: 0.7437962293624878 | f1: 0.816154420375824 | auc: 0.8038568496704102 | treshold: 0.44
Test: accuracy: 0.7392576336860657 | f1: 0.81426233

/opt/conda/envs/myenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


Val update: epoch: 28 |accuracy: 0.7944661974906921 | f1: 0.8455414772033691 | auc: 0.8591319918632507 | treshold: 0.4
Test: accuracy: 0.7851045727729797 | f1: 0.8391755223274231 | auc: 0.851215124130249 | 
Val update: epoch: 29 |accuracy: 0.772698700428009 | f1: 0.8439544439315796 | auc: 0.859998345375061 | treshold: 0.38
Test: accuracy: 0.7725788950920105 | f1: 0.8446096777915955 | auc: 0.8515419960021973 | 
Val update: epoch: 32 |accuracy: 0.7918782830238342 | f1: 0.8492704629898071 | auc: 0.8609446287155151 | treshold: 0.41000000000000003
Test: accuracy: 0.7870874404907227 | f1: 0.8463967442512512 | auc: 0.8534929752349854 | 
Val update: epoch: 35 |accuracy: 0.796352744102478 | f1: 0.8502845168113708 | auc: 0.8615210056304932 | treshold: 0.4
Test: accuracy: 0.7876919507980347 | f1: 0.8445907831192017 | auc: 0.8536247611045837 | 
Val update: epoch: 39 |accuracy: 0.7934504151344299 | f1: 0.8512765169143677 | auc: 0.8636046648025513 | treshold: 0.42000000000000004
Test: accuracy: 0.78

In [11]:
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items